In [1]:
!head lyrl2004_tokens_train.dat

.I 2286
.W
recov recov recov recov excit excit bring mexic mexic mexic mexic mexic mexic mexic mexic mexic mexic market market market market market market market life emerg evident evident econom econom
econom econom back back back track buzz tuesday tuesday tuesday tuesday tuesday stock stock stock clos record record high high high interest interest interest interest rate rate rate rate rate rate
rate month low low low stag begin year ahead term term term term fundament fundament fundament matthew hickm lehm brother york point point point hist hist fall fall fall view etch
mind invest cris decemb free peso peso peso stubborn week week quart gross domest domest produc report percent percent percent strong strong strong anal anal expect expect expect expect expect govern
treasur bill cete cete second second fell level jan main pric index rally volum frenz million shar confound strength end long long contract drop benchmark auction remain stead feder reserv refrain
rais short attract off

In [16]:
train_res = {}
with open('lyrl2004_tokens_train.dat', 'r') as f:
    prev_i = ''
    cur_line = ''
    for line in f:
        if '.I' in line:
            i = line.strip().split()[1]
            if prev_i == '':
                prev_i = i
            if cur_line != '':
                train_res[prev_i] = cur_line.strip()
                cur_line = ''
                prev_i = i
            continue
        if '.W' in line:
            continue
        cur_line += line.strip() + ' '
    train_res[prev_i] = cur_line.strip()

In [21]:
len(train_res)

23149

In [20]:
!head lyrl2004_tokens_test_pt0.dat

.I 26151
.W
socc colomb colomb colomb colomb beat beat chil chil chil chil world world world cup cup cup qualif qualif qualif qualif halftim south south americ americ match sunday scor faustin
asprill st minut minut jorg bermudez ivan zamoran penalt attend group stand tabulat play won draw lost goal goal point ecuador argentin boliv paraguay uruguay peru venezuel note top final franc
brazil automat hold

.I 26152
.W
world world world world qualif qualif sunday minut minut won hold athlet time time time time time lucky lucky komen komen komen komen break break record record record daniel keny
keny keny keny keny keny made shat noureddin morcel morcel morcel morcel morcel morcel morcel met met met met met met second second second second intern meet year year fail fail


In [169]:
test_res = {}
for k in range(4):
    with open('lyrl2004_tokens_test_pt'+str(k)+'.dat', 'r') as f:
        prev_i = ''
        cur_line = ''
        for line in f:
            if '.I' in line:
                i = line.strip().split()[1]
                if prev_i == '':
                    prev_i = i
                if cur_line != '':
                    test_res[prev_i] = cur_line.strip()
                    cur_line = ''
                    prev_i = i
                continue
            if '.W' in line:
                continue
            cur_line += line.strip() + ' '
        test_res[prev_i] = cur_line.strip()

In [171]:
for k, v in test_res.items():
    print(k, v)
    break

26151 socc colomb colomb colomb colomb beat beat chil chil chil chil world world world cup cup cup qualif qualif qualif qualif halftim south south americ americ match sunday scor faustin asprill st minut minut jorg bermudez ivan zamoran penalt attend group stand tabulat play won draw lost goal goal point ecuador argentin boliv paraguay uruguay peru venezuel note top final franc brazil automat hold


In [30]:
import numpy as np

val_ids = np.random.choice(list(train_res.keys()), 2316, replace=False)

In [32]:
val_ids = list(val_ids)

In [35]:
train_ids = []
for k in train_res:
    if k not in val_ids:
        train_ids.append(k)

In [37]:
len(train_ids)

20833

In [38]:
len(val_ids)

2316

In [172]:
labels = {}
with open('rcv1-v2.topics.qrels', 'r') as f:
    for line in f:
        label, key, _ = line.strip().split()
        if key not in labels:
            labels[key] = []
        labels[key].append(label.lower())

In [181]:
hier = {}
with open('HiAGM/data/rcv1.taxonomy') as f:
    for line in f:
        tmp = line.strip().lower().split('\t')
        for i in range(len(tmp)):
            if tmp[i] not in hier:
                hier[tmp[i]] = []
            if i != 0:
                hier[tmp[0]].append(tmp[i])

In [182]:
def get_level(node, dct, lvl=0):
    dct[node] = lvl
    for elem in hier[node]:
        get_level(elem, dct, lvl+1)

In [183]:
labels_levels = {}
get_level('root', labels_levels)

In [184]:
labels_sorted = {}
for k, v in labels.items():
    labels_sorted[k] = sorted(v, key=lambda x: labels_levels[x])

NameError: name 'labels' is not defined

In [63]:
!mkdir rcv1_v2_data

In [71]:
with open('rcv1_v2_data/train.src', 'w') as f:
    with open('rcv1_v2_data/train.tgt', 'w') as g:
        for k, v in train_res.items():
            if k in train_ids:
                f.write(v + '\n')
                g.write(' '.join(labels_sorted[k]) + '\n')

In [72]:
!head rcv1_v2_data/train.src

recov recov recov recov excit excit bring mexic mexic mexic mexic mexic mexic mexic mexic mexic mexic market market market market market market market life emerg evident evident econom econom econom econom back back back track buzz tuesday tuesday tuesday tuesday tuesday stock stock stock clos record record high high high interest interest interest interest rate rate rate rate rate rate rate month low low low stag begin year ahead term term term term fundament fundament fundament matthew hickm lehm brother york point point point hist hist fall fall fall view etch mind invest cris decemb free peso peso peso stubborn week week quart gross domest domest produc report percent percent percent strong strong strong anal anal expect expect expect expect expect govern treasur bill cete cete second second fell level jan main pric index rally volum frenz million shar confound strength end long long contract drop benchmark auction remain stead feder reserv refrain rais short attract off robust ret

In [74]:
!head rcv1_v2_data/train.tgt

ecat mcat e11 m11 m12
ccat c24
ccat c15 c151
mcat m11 m12 m13 m14 m132
ccat c22
ecat e14
ccat gcat c12 gcrim
ccat ecat gcat c13 c31 c42 e41 gjob
ecat mcat e12 m13 m131
ccat gcat c12 gcrim


In [75]:
with open('rcv1_v2_data/valid.src', 'w') as f:
    with open('rcv1_v2_data/valid.tgt', 'w') as g:
        for k, v in train_res.items():
            if k in val_ids:
                f.write(v + '\n')
                g.write(' '.join(labels_sorted[k]) + '\n')

In [77]:
!wc -l rcv1_v2_data/train.src

20833 rcv1_v2_data/train.src


In [177]:
with open('rcv1_v2_data/test.src', 'w') as f:
    with open('rcv1_v2_data/test.tgt', 'w') as g:
        for k, v in test_res.items():
            f.write(v + '\n')
            g.write(' '.join(labels_sorted[k]) + '\n')

In [178]:
!wc -l rcv1_v2_data/test.tgt

781265 rcv1_v2_data/test.tgt


In [179]:
!wc -l rcv1_v2_data/test.src

781265 rcv1_v2_data/test.src


In [83]:
!tail rcv1_v2_data/test.tgt

gcat gent
ccat gcat c23 ghea
mcat m11
ccat ecat gcat c42 e41 gjob
ccat ecat gcat c42 e41 gjob
ccat ecat gcat c42 e41 gjob
gcat gjob
ccat ecat gcat c42 e41 gjob
ccat c21 c24
gcat gdis genv


# Exp

In [157]:
cnt = 0
with open('rcv1_v2_data/test.tgt', 'r') as f:
    for line in f:
        temp_counter = {0: 0, 1: 0, 2: 0, 3: 0, 4: 0}
        tmp = line.strip().split()
        flag = False
        for elem in tmp:
            if temp_counter[labels_levels[elem]] == 1:
                flag = True
                break
            temp_counter[labels_levels[elem]] += 1
        if not flag:
            cnt += 1
#             print(temp_counter, line)
cnt

531742

In [168]:
!wc -l rcv1_v2_data/test.tgt

781265 rcv1_v2_data/test.tgt


In [164]:
cnt = 0
res = set()
with open('rcv1_v2_data/test.src', 'r') as f:
    for line in f:
        cnt += 1
#             print(temp_counter, line)
cnt

0

In [99]:
cnt = 0
with open('rcv1_v2_data/train.tgt', 'r') as f:
    for line in f:
        temp_counter = {1: 0, 2: 0, 3: 0, 4: 0}
        tmp = line.strip().split()
        flag = False
        for elem in tmp:
            temp_counter[labels_levels[elem]] += 1
        prev_v = 9999
        for k, v in temp_counter.items():
            if v > prev_v:
                flag = True
                break
            prev_v = v
        if not flag:
            cnt += 1
#             print(temp_counter, line)
cnt

16285

In [199]:
cnt = 0
res_train = []
with open('rcv1_v2_data/train.tgt', 'r') as f:
    i = -1
    for line in f:
        i += 1
        temp_counter = {1: 0, 2: 0, 3: 0, 4: 0}
        temp_levels = {1: '', 2: '', 3: '', 4: ''}
        tmp = line.strip().split()
        flag = False
        for elem in tmp:
            if temp_counter[labels_levels[elem]] == 1:
                flag = True
                break
            temp_counter[labels_levels[elem]] += 1
            temp_levels[labels_levels[elem]] = elem
        if not flag:
            for k, v in temp_counter.items():
                if v == 0:
                    if len(hier[temp_levels[k-1]]) > 0:
                        flag = True
                    break
            if not flag:
                res_train.append(i)
                cnt += 1
cnt

11377

In [200]:
cnt = 0
res_val = []
with open('rcv1_v2_data/valid.tgt', 'r') as f:
    i = -1
    for line in f:
        i += 1
        temp_counter = {1: 0, 2: 0, 3: 0, 4: 0}
        temp_levels = {1: '', 2: '', 3: '', 4: ''}
        tmp = line.strip().split()
        flag = False
        for elem in tmp:
            if temp_counter[labels_levels[elem]] == 1:
                flag = True
                break
            temp_counter[labels_levels[elem]] += 1
            temp_levels[labels_levels[elem]] = elem
        if not flag:
            for k, v in temp_counter.items():
                if v == 0:
                    if len(hier[temp_levels[k-1]]) > 0:
                        flag = True
                    break
            if not flag:
                res_val.append(i)
                cnt += 1
cnt

1274

In [212]:
cnt = 0
res_test = set()
with open('rcv1_v2_data/test.tgt', 'r') as f:
    i = -1
    for line in f:
        i += 1
        temp_counter = {1: 0, 2: 0, 3: 0, 4: 0}
        temp_levels = {1: '', 2: '', 3: '', 4: ''}
        tmp = line.strip().split()
        flag = False
        for elem in tmp:
            if temp_counter[labels_levels[elem]] == 1:
                flag = True
                break
            temp_counter[labels_levels[elem]] += 1
            temp_levels[labels_levels[elem]] = elem
        if not flag:
            for k, v in temp_counter.items():
                if v == 0:
                    if len(hier[temp_levels[k-1]]) > 0:
                        flag = True
                    break
            if not flag:
                res_test.add(i)
                cnt += 1
cnt

430940

In [349]:
res = {}
with open('rcv1.taxonomy') as f:
    for line in f:
        tmp = line.strip().split('\t')
        for i in range(len(tmp)):
            if tmp[i] not in res:
                res[tmp[i]] = []
            if i != 0:
                res[tmp[0]].append(tmp[i])
                
key2i = {}
for i, elem in enumerate(list(res.keys())):
    key2i[elem] = i

N = len(key2i) - 1

In [352]:
import tqdm

with open('rcv1_v2_data/train.tgt', 'r') as f:
    with open('rcv1_v2_data/train.src', 'r') as g:
        with open('train_docbert.rcv', 'w') as fg:
            tgt_readlines = f.readlines()
            src_readlines = g.readlines()
            for i in tqdm.tqdm(range(len(src_readlines))):
                label = ['0'] * N
                s = src_readlines[i]
                t = tgt_readlines[i].strip().split()
                for e in t:
                    label[key2i[e.upper()] - 1] = '1'
                fg.write(''.join(label) + '\t' + s)

100%|██████████| 20833/20833 [00:02<00:00, 8613.30it/s]


In [353]:
with open('rcv1_v2_data/valid.tgt', 'r') as f:
    with open('rcv1_v2_data/valid.src', 'r') as g:
        with open('valid_docbert.rcv', 'w') as fg:
            tgt_readlines = f.readlines()
            src_readlines = g.readlines()
            for i in tqdm.tqdm(range(len(src_readlines))):
                label = ['0'] * N
                s = src_readlines[i]
                t = tgt_readlines[i].strip().split()
                for e in t:
                    label[key2i[e.upper()] - 1] = '1'
                fg.write(''.join(label) + '\t' + s)

100%|██████████| 2316/2316 [00:00<00:00, 224031.55it/s]


In [354]:
with open('rcv1_v2_data/test.tgt', 'r') as f:
    with open('rcv1_v2_data/test.src', 'r') as g:
        with open('test_docbert.rcv', 'w') as fg:
            tgt_readlines = f.readlines()
            src_readlines = g.readlines()
            for i in tqdm.tqdm(range(len(src_readlines))):
                label = ['0'] * N
                s = src_readlines[i]
                t = tgt_readlines[i].strip().split()
                for e in t:
                    label[key2i[e.upper()] - 1] = '1'
                fg.write(''.join(label) + '\t' + s)

100%|██████████| 781265/781265 [00:03<00:00, 258348.66it/s]


In [203]:
res = {}
with open('rcv1.taxonomy') as f:
    for line in f:
        tmp = line.strip().split('\t')
        for i in range(len(tmp)):
            if tmp[i] not in res:
                res[tmp[i]] = []
            if i != 0:
                res[tmp[0]].append(tmp[i])

In [205]:
key2i = {}
for i, elem in enumerate(list(res.keys())):
    key2i[elem] = i

In [206]:
N = len(key2i)

In [207]:
from tqdm import tqdm

with open('rcv1_v2_data/train.tgt', 'r') as f:
    with open('rcv1_v2_data/train.src', 'r') as g:
        with open('train_docbert_hiermax.rcv', 'w') as fg:
            tgt_readlines = f.readlines()
            src_readlines = g.readlines()
            for i in tqdm(range(len(src_readlines))):
                if i not in res_train:
                    continue
                label = ['0'] * N
                label[0] = '1'
                s = src_readlines[i]
                t = tgt_readlines[i].strip().split()
                for e in t:
                    label[key2i[e.upper()]] = '1'
                fg.write(''.join(label) + '\t' + s)

100%|██████████| 20833/20833 [00:02<00:00, 9934.40it/s]


In [208]:
!wc -l train_docbert_hiermax.rcv

11377 train_docbert_hiermax.rcv


In [209]:
with open('rcv1_v2_data/valid.tgt', 'r') as f:
    with open('rcv1_v2_data/valid.src', 'r') as g:
        with open('valid_docbert_hiermax.rcv', 'w') as fg:
            tgt_readlines = f.readlines()
            src_readlines = g.readlines()
            for i in tqdm(range(len(src_readlines))):
                if i not in res_val:
                    continue
                label = ['0'] * N
                label[0] = '1'
                s = src_readlines[i]
                t = tgt_readlines[i].strip().split()
                for e in t:
                    if e == 'm12ccat':
                        label[key2i['M12']] = '1'
                        label[key2i['CCAT']] = '1'
                        continue
                    label[key2i[e.upper()]] = '1'
                fg.write(''.join(label) + '\t' + s)

100%|██████████| 2316/2316 [00:00<00:00, 72003.62it/s]


In [210]:
!wc -l valid_docbert_hiermax.rcv

1274 valid_docbert_hiermax.rcv


In [213]:
with open('rcv1_v2_data/test.tgt', 'r') as f:
    with open('rcv1_v2_data/test.src', 'r') as g:
        with open('test_docbert_hiermax.rcv', 'w') as fg:
            tgt_readlines = f.readlines()
            src_readlines = g.readlines()
            for i in tqdm(range(len(src_readlines))):
                if i not in res_test:
                    continue
                label = ['0'] * N
                label[0] = '1'
                s = src_readlines[i]
                t = tgt_readlines[i].strip().split()
                for e in t:
                    label[key2i[e.upper()]] = '1'
                fg.write(''.join(label) + '\t' + s)

100%|██████████| 781265/781265 [00:01<00:00, 457720.91it/s]


In [216]:
!wc -l test_docbert_hiermax.rcv

430940 test_docbert_hiermax.rcv


In [ ]:
cnt = 0
res = set()
with open('rcv1_v2_data/train.tgt', 'r') as f:
    for line in f:
        temp_counter = {1: 0, 2: 0, 3: 0, 4: 0}
        tmp = line.strip().split()
        flag = False
        for elem in tmp:
            if temp_counter[labels_levels[elem]] == 1:
                flag = True
                break
            temp_counter[labels_levels[elem]] += 1
        if not flag:
            for k, v in temp_counter.items():
                if v == 0:
                    if len(hier[temp_levels[k-1]]) > 0:
                        flag = True
                    break
            if not flag:
                res.add(line.strip())
                cnt += 1
#                 print(temp_counter, line)
cnt

In [156]:
for elem in res1:
    if elem not in res:
        print(elem)

gcat gobit
gcat g15 g151
gcat g15 g158
gcat gfas
gcat g15 g156
gcat g15 g152
gcat g15 g157
ecat e31 e312
gcat g15 g153


In [153]:
res1 = res

In [100]:
hier

{'root': ['ccat', 'ecat', 'gcat', 'mcat'],
 'ccat': ['c11',
  'c12',
  'c13',
  'c14',
  'c15',
  'c16',
  'c17',
  'c18',
  'c21',
  'c22',
  'c23',
  'c24',
  'c31',
  'c32',
  'c33',
  'c34',
  'c41',
  'c42'],
 'ecat': ['e11',
  'e12',
  'e13',
  'e14',
  'e21',
  'e31',
  'e41',
  'e51',
  'e61',
  'e71'],
 'gcat': ['g15',
  'gcrim',
  'gdef',
  'gdip',
  'gdis',
  'gent',
  'genv',
  'gfas',
  'ghea',
  'gjob',
  'gmil',
  'gobit',
  'godd',
  'gpol',
  'gpro',
  'grel',
  'gsci',
  'gspo',
  'gtour',
  'gvio',
  'gvote',
  'gwea',
  'gwelf'],
 'mcat': ['m11', 'm12', 'm13', 'm14'],
 'c11': [],
 'c12': [],
 'c13': [],
 'c14': [],
 'c15': ['c151', 'c152'],
 'c16': [],
 'c17': ['c171', 'c172', 'c173', 'c174'],
 'c18': ['c181', 'c182', 'c183'],
 'c21': [],
 'c22': [],
 'c23': [],
 'c24': [],
 'c31': ['c311', 'c312', 'c313'],
 'c32': [],
 'c33': ['c331'],
 'c34': [],
 'c41': ['c411'],
 'c42': [],
 'c151': ['c1511'],
 'c152': [],
 'c1511': [],
 'c171': [],
 'c172': [],
 'c173': [],
 'c

In [84]:
labels_levels

{'root': 0,
 'ccat': 1,
 'c11': 2,
 'c12': 2,
 'c13': 2,
 'c14': 2,
 'c15': 2,
 'c151': 3,
 'c1511': 4,
 'c152': 3,
 'c16': 2,
 'c17': 2,
 'c171': 3,
 'c172': 3,
 'c173': 3,
 'c174': 3,
 'c18': 2,
 'c181': 3,
 'c182': 3,
 'c183': 3,
 'c21': 2,
 'c22': 2,
 'c23': 2,
 'c24': 2,
 'c31': 2,
 'c311': 3,
 'c312': 3,
 'c313': 3,
 'c32': 2,
 'c33': 2,
 'c331': 3,
 'c34': 2,
 'c41': 2,
 'c411': 3,
 'c42': 2,
 'ecat': 1,
 'e11': 2,
 'e12': 2,
 'e121': 3,
 'e13': 2,
 'e131': 3,
 'e132': 3,
 'e14': 2,
 'e141': 3,
 'e142': 3,
 'e143': 3,
 'e21': 2,
 'e211': 3,
 'e212': 3,
 'e31': 2,
 'e311': 3,
 'e312': 3,
 'e313': 3,
 'e41': 2,
 'e411': 3,
 'e51': 2,
 'e511': 3,
 'e512': 3,
 'e513': 3,
 'e61': 2,
 'e71': 2,
 'gcat': 1,
 'g15': 2,
 'g151': 3,
 'g152': 3,
 'g153': 3,
 'g154': 3,
 'g155': 3,
 'g156': 3,
 'g157': 3,
 'g158': 3,
 'g159': 3,
 'gcrim': 2,
 'gdef': 2,
 'gdip': 2,
 'gdis': 2,
 'gent': 2,
 'genv': 2,
 'gfas': 2,
 'ghea': 2,
 'gjob': 2,
 'gmil': 2,
 'gobit': 2,
 'godd': 2,
 'gpol': 2,
 'gpro

# WOS

In [217]:
!mv HiAGM/data/Data.xlsx metadata.txt

In [230]:
!wc -l wos_total.json

25820 wos_total.json


In [252]:
!python3 HiAGM/data/preprocess_wos.py

{'0-12': ['Symbolic computation'], '5-2': ["Alzheimer's Disease"], '4-7': ['Green Building'], '1-10': ['Electric motor', 'Satellite radio', 'Single-phase electric power'], '5-43': ["Parkinson's Disease"], '0-0': ['Computer vision'], '6-0': ['Molecular biology'], '3-4': ['Fluid mechanics'], '2-13': ['Prenatal development'], '5-51': ['Sprains and Strains'], '6-5': ['Enzymology'], '6-8': ['Southern blotting'], '5-38': ['Cancer'], '6-7': ['Northern blotting'], '5-50': ['Sports Injuries'], '5-48': ['Senior Health'], '0-5': ['Computer graphics'], '1-4': ['Digital control'], '6-2': ['Human Metabolism'], '6-6': ['Polymerase chain reaction'], '5-36': ['Multiple Sclerosis'], '0-4': ['Operating systems'], '1-7': ['Microcontroller'], '2-17': ['Attention'], '6-3': ['Immunology'], '6-4': ['Genetics'], '4-4': ['Water Pollution', 'Bamboo as a Building Material', 'Underwater Windmill'], '3-1': ['Hydraulics'], '5-24': ['Hepatitis C'], '5-16': ['Weight Loss'], '0-1': ['Machine learning'], '5-31': ['Low T

In [327]:
!head -1 HiAGM/data/wos_test.json

{"token": ["high", "degree", "uncertainty", "associated", "emission", "inventory", "china", "tends", "degrade", "performance", "chemical", "transport", "models", "predicting", "pm2", "concentrations", "especially", "daily", "basis", "study", "novel", "machine", "learning", "algorithm", "geographically", "weighted", "gradient", "boosting", "machine", "gw", "gbm", "developed", "improving", "gbm", "building", "spatial", "smoothing", "kernels", "weigh", "loss", "function", "modification", "addressed", "spatial", "nonstationarity", "relationships", "pm2", "concentrations", "predictor", "variables", "aerosol", "optical", "depth", "aod", "meteorological", "conditions", "gw", "gbm", "also", "overcame", "estimation", "bias", "pm2", "concentrations", "due", "missing", "aod", "retrievals", "thus", "potentially", "improved", "subsequent", "exposure", "analyses", "gw", "gbm", "showed", "good", "performance", "predicting", "daily", "pm2", "concentrations", "76", "rmse", "23", "even", "partially", "m

In [235]:
!head wos.taxnomy

Root	CS	Medical	Civil	ECE	biochemistry	MAE	Psychology
CS	Symbolic computation	Computer vision	Computer graphics	Operating systems	Machine learning	Data structures	network security	Image processing	Parallel computing	Distributed computing	Algorithm design	Computer programming	Relational databases	Software engineering	Bioinformatics	Cryptography	Structured Storage
Medical	Alzheimer's Disease	Parkinson's Disease	Sprains and Strains	Cancer	Sports Injuries	Senior Health	Multiple Sclerosis	Hepatitis C	Weight Loss	Low Testosterone	Fungal Infection	Diabetes	Parenting	Birth Control	Heart Disease	Allergies	Menopause	Emergency Contraception	Skin Care	Myelofibrosis	Hypothyroidism	Headache	Overactive Bladder	Irritable Bowel Syndrome	Polycythemia Vera	Atrial Fibrillation	Smoking Cessation	Lymphoma	Asthma	Bipolar Disorder	Crohn's Disease	Idiopathic Pulmonary Fibrosis	Mental Health	Dementia	Rheumatoid Arthritis	Osteoporosis	Medicare	Psoriatic Arthritis	Addiction	Atopic Dermatitis	Digestive Health	Heal

In [253]:
!mv wos_train.json HiAGM/data/

In [254]:
!mv wos_test.json HiAGM/data/
!mv wos_val.json HiAGM/data/
!mv wos_total.json HiAGM/data/

In [255]:
!mv wos.taxnomy HiAGM/data

In [240]:
!python3 HiAGM/helper/hierarchy_tree_statistic.py HiAGM/config/rcv1-v2-Copy1.json

Traceback (most recent call last):
  File "HiAGM/helper/hierarchy_tree_statistic.py", line 5, in <module>
    from models.structure_model.tree import Tree
ModuleNotFoundError: No module named 'models'


Exception: Process exited with code 1

In [243]:
!cd HiAGM && python3 helper/hierarchy_tree_statistic.py HiAGM/config/rcv1-v2-Copy1.json

Traceback (most recent call last):
  File "helper/hierarchy_tree_statistic.py", line 5, in <module>
    from models.structure_model.tree import Tree
ModuleNotFoundError: No module named 'models'


Exception: Process exited with code 1

In [256]:
!cd HiAGM && python3 new.py config/rcv1-v2-Copy1.json

{'Root': ['CS', 'Medical', 'Civil', 'ECE', 'biochemistry', 'MAE', 'Psychology'], 'CS': ['Symbolic computation', 'Computer vision', 'Computer graphics', 'Operating systems', 'Machine learning', 'Data structures', 'network security', 'Image processing', 'Parallel computing', 'Distributed computing', 'Algorithm design', 'Computer programming', 'Relational databases', 'Software engineering', 'Bioinformatics', 'Cryptography', 'Structured Storage'], 'Medical': ["Alzheimer's Disease", "Parkinson's Disease", 'Sprains and Strains', 'Cancer', 'Sports Injuries', 'Senior Health', 'Multiple Sclerosis', 'Hepatitis C', 'Weight Loss', 'Low Testosterone', 'Fungal Infection', 'Diabetes', 'Parenting', 'Birth Control', 'Heart Disease', 'Allergies', 'Menopause', 'Emergency Contraception', 'Skin Care', 'Myelofibrosis', 'Hypothyroidism', 'Headache', 'Overactive Bladder', 'Irritable Bowel Syndrome', 'Polycythemia Vera', 'Atrial Fibrillation', 'Smoking Cessation', 'Lymphoma', 'Asthma', 'Bipolar Disorder', "Cro

In [249]:
!mv HiAGM/data/new.py HiAGM/

In [258]:
!head HiAGM/data/wos_prob.json

{"Root": {"CS": 0, "Medical": 0, "Civil": 0, "ECE": 0, "biochemistry": 0, "MAE": 0, "Psychology": 0}, "CS": {"Symbolic computation": 0.06007269944518843, "Computer vision": 0.06542950066960014, "Computer graphics": 0.06255978572795101, "Operating systems": 0.05701167017409602, "Machine learning": 0.06160321408073463, "Data structures": 0.061411899751291354, "network security": 0.06906447292902237, "Image processing": 0.06409030036349721, "Parallel computing": 0.06944710158790893, "Distributed computing": 0.06179452841017791, "Algorithm design": 0.05815955615075568, "Computer programming": 0.06466424335182704, "Relational databases": 0.0587334991390855, "Software engineering": 0.06389898603405393, "Bioinformatics": 0.05682035584465275, "Cryptography": 0.0587334991390855, "Structured Storage": 0.006504687201071359}, "Medical": {"Alzheimer's Disease": 0.02478146482220147, "Parkinson's Disease": 0.020453195281337516, "Sprains and Strains": 0.009505219383858099, "Cancer": 0.0249512008826275

# WOS

In [328]:
res = {}
with open('HiAGM/data/wos.taxnomy') as f:
    for line in f:
        tmp = line.strip().split('\t')
        for i in range(len(tmp)):
            if tmp[i] not in res:
                res[tmp[i]] = []
            if i != 0:
                res[tmp[0]].append(tmp[i])

In [330]:
key2i = {}
for i, elem in enumerate(list(res.keys())):
    key2i[elem] = i
N = len(key2i) - 1

In [346]:
import json

with open('HiAGM/data/wos_train.json', 'r') as f:
    with open('train_docbert.wos', 'w') as fg:
        lines = f.readlines()
        for i in tqdm(range(len(lines))):
            label = ['0'] * N
            line = json.loads(lines[i])
            for e in line['label']:
                label[key2i[e] - 1] = '1'
            fg.write(''.join(label) + '\t' + ' '.join(line['token']) + '\n')

100%|██████████| 30070/30070 [00:03<00:00, 7553.36it/s]


In [347]:
with open('HiAGM/data/wos_val.json', 'r') as f:
    with open('valid_docbert.wos', 'w') as fg:
        lines = f.readlines()
        for i in tqdm(range(len(lines))):
            label = ['0'] * N
            line = json.loads(lines[i])
            for e in line['label']:
                label[key2i[e] - 1] = '1'
            fg.write(''.join(label) + '\t' + ' '.join(line['token']) + '\n')

100%|██████████| 7518/7518 [00:00<00:00, 48645.48it/s]


In [348]:
with open('HiAGM/data/wos_test.json', 'r') as f:
    with open('test_docbert.wos', 'w') as fg:
        lines = f.readlines()
        for i in tqdm(range(len(lines))):
            label = ['0'] * N
            line = json.loads(lines[i])
            for e in line['label']:
                label[key2i[e] - 1] = '1'
            fg.write(''.join(label) + '\t' + ' '.join(line['token']) + '\n')

100%|██████████| 9397/9397 [00:00<00:00, 47329.84it/s]


In [345]:
!rm test_docbert_hiermax.wos

In [342]:
!head valid_docbert.wos

In [276]:
res = {}
with open('HiAGM/data/wos.taxnomy') as f:
    for line in f:
        tmp = line.strip().split('\t')
        for i in range(len(tmp)):
            if tmp[i] not in res:
                res[tmp[i]] = []
            if i != 0:
                res[tmp[0]].append(tmp[i])

In [277]:
key2i = {}
for i, elem in enumerate(list(res.keys())):
    key2i[elem] = i
N = len(key2i)

In [318]:
with open('HiAGM/data/wos_train.json', 'r') as f:
    with open('train_t5_wos.csv', 'w') as fg:
        fg.write('ID,TEXT,'+','.join(list(key2i.keys())[1:])+'\n')
        lines = f.readlines()
        for i in tqdm(range(len(lines))):
            label = ['0'] * (N - 1)
            line = json.loads(lines[i])
            for e in line['label']:
                label[key2i[e] - 1] = '1'
            fg.write(str(i)+','+' '.join(line['token'])+','+','.join(label)+'\n')

100%|██████████| 30070/30070 [00:00<00:00, 49891.54it/s]


In [323]:
with open('HiAGM/data/wos_test.json', 'r') as f:
    with open('test_t5_wos.csv', 'w') as fg:
        fg.write('ID,TEXT,'+','.join(list(key2i.keys())[1:])+'\n')
        lines = f.readlines()
        for i in tqdm(range(len(lines))):
            label = ['0'] * (N - 1)
            line = json.loads(lines[i])
            for e in line['label']:
                label[key2i[e] - 1] = '1'
            fg.write(str(i)+','+' '.join(line['token'])+','+','.join(label)+'\n')

100%|██████████| 9397/9397 [00:00<00:00, 49414.17it/s]


In [325]:
with open('HiAGM/data/wos_val.json', 'r') as f:
    with open('val_t5_wos.csv', 'w') as fg:
        fg.write('ID,TEXT,'+','.join(list(key2i.keys())[1:])+'\n')
        lines = f.readlines()
        for i in tqdm(range(len(lines))):
            label = ['0'] * (N - 1)
            line = json.loads(lines[i])
            for e in line['label']:
                label[key2i[e] - 1] = '1'
            fg.write(str(i)+','+' '.join(line['token'])+','+','.join(label)+'\n')

100%|██████████| 7518/7518 [00:00<00:00, 48153.63it/s]
